<img src="images/inmas.png" width=130x align='right' />

# Notebook 18 - SciPy Basics

Material covered in this notebook:

- How to solve a system of ODEs using SciPy
- How to solve a linear programming problem
- How to use special functions in the SciPy module

#### Prerequisite
Notebook 17

#### Housekeeping

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### What is SciPy?

- `SciPy` in Python is an open-source library of algorithms designed to solve mathematical problems
- It builds on top of the low-level `NumPy` framework for multidimensional arrays
- It provides a large number of higher-level scientific algorithms grouped in submodules


### SciPy has many submodules

<table>
<tr>
<td>
    
    Special functions (scipy.special)
    Integration (scipy.integrate)
    Optimization (scipy.optimize)
    Interpolation (scipy.interpolate)
    Fourier Transforms (scipy.fftpack)
    Signal Processing (scipy.signal)
    
</td>
<td>
    
    Linear Algebra (scipy.linalg)
    Sparse Eigenvalue Problems (scipy.sparse)
    Statistics (scipy.stats)
    Multi-dimensional image processing (scipy.ndimage)
    File IO (scipy.io)
    
</td>
</tr>
</table>

Each of these submodules provides a number of functions and classes that can be used to solve problems in their respective topics

### SciPy can solve ordinary differential equations (ODE)

- `SciPy` provides two different ways to solve ODEs:
    - a procedural approach based on the function `odeint`
    - an object-oriented API based on the class `ode`
- For non-OOP users, `odeint` is easier to get started with, but the `ode` class offers some finer level of control

Here we will use the `odeint` functions from submodule `scipy.integrate`

In [ ]:
from scipy.integrate import odeint

### Formulation a system of ODE's
The standard form of a system of ODE's is:

$y′=f(y, t)$

where

$y=[y_1(t),y_2(t),...,y_n(t)]$

and $f$ is some function that gives the derivatives of the functions $y_i(t)$

To solve an initial-value problem we need to know the function $f$ and initial conditions, $y(0)$

### Solving an ODE system in SciPy

For solving in Python, all we need is to define the derivative function `f()` and build the array of initial values `y0` and  discretized time coordinates `t`
 
we can then use the `odeint` function as:

```
yt = odeint(f, y0, t)
```

where `yt` is an array with one row for each point in time in `t`, and where each column corresponds to a solution `y_i(t)` at that point in time

Let's look at some examples

### Predator-prey equations

- One fist example considers a two-species predator-prey equations
- These equations are named the Lotka-Volterra equations
    - more details [here](https://en.wikipedia.org/wiki/Lotka–Volterra_equations)
- For two species, they involve 4 coefficients and can be written as:

> $\dot{x} = \alpha x-\beta xy$ <br>
> $\dot{y} = \delta xy -\gamma y$

where $\dot{x}$ is the time derivative of $x$.

Predators $y$ thrive when $x$ population is large, while preys $x$' growth decreases if predators are numerous

### Coding the derivative in Python

In [ ]:
alpha = 1.
beta = 0.1
delta = 0.07
gamma = 1.5

def derivative(X, t):
    '''The right-hand side of the ODE.'''
    x, y = X
    xdot = x * (alpha - beta * y)
    ydot = y * (delta * x - gamma)
    return np.array([xdot, ydot])

### Defining initial conditions and time discretization
We will now define an initial population of 10 rabbits and 5 foxes:

In [ ]:
X0 = np.array([10, 5]) # 10 preys and 5 predators

We choose to model for 15 periods discretized over 1000 points:

In [ ]:
t = np.linspace(0, 15,  1000) 

### Integrating the  system of ODEs
A simple call integrates this system of ODEs:

In [ ]:
X = odeint(derivative, X0, t)

The results in time are in rows. We need to transpose to split time series by preys and predators:

In [ ]:
prey, predator = X.transpose()

### Plotting the results with matplotlib

In [ ]:
f1 = plt.figure()
plt.plot(t, prey, 'r-', label='Preys')
plt.plot(t, predator  , 'b-', label='Predators')
plt.grid()
plt.legend(loc='best')
plt.xlabel('time')
plt.ylabel('population')
plt.title('Evolution of prey and predator populations');

### SciPy can also solve many types of optimization problems
- **minimize()** for minimizing a scalar function of one or more variables (local minimum)
    - many other methods to deal with global optimization
- **linprog()** for solving linear programming problems
     - **milp()** for solving linear programming problems with mixed integers
- **leastsq()** for minimizing the sum of squares of a set of equations
- **curve_fit** for non-linear least-squares fit to a function


### A simple linear programming example
- Company produces 4 products priced at \\$12, \\$14, \\$25, and \\$40 respectively
- Production line cannot produce more that 50 units per day
- Producing these units costs 3, 2, 1, 0 units of material A and 0, 2, 2, 5 units of material B, respectively
- Supply chain limits consumption of 100 units of material A and 90 units of material B per day

Maximize profits:
$$
\max 12x_1 + 14 x_2 + 25 x_3 + 40 x_4\\
\text{subject to:}\hspace{1cm} x_1 + x_2 + x_3 + x_4 \le 50 \\
\hspace{2.2cm} 3 x_1 + 2 x_2 +  x_3 \le 100 \\
\hspace{2.2cm} 2 x_2 + 2 x_3 + 5 x_4 \le 90
$$

### Converting to maximize $c \cdot x$ subject to $Ax \le b$
$$
\max 12x_1 + 14 x_2 + 25 x_3 + 40 x_4\\
\text{subject to:}\hspace{1cm} x_1 + x_2 + x_3 + x_4 \le 50 \\
\hspace{2.2cm} 3 x_1 + 2 x_2 +  x_3 \le 100 \\
\hspace{2.2cm} 2 x_2 + 2 x_3 + 5 x_4 \le 90,
$$
becomes
$$
c = (12, 14, 25, 40),\\
A = 
\begin{pmatrix}
1 & 1 & 1 & 1 \\
3 & 2 & 1 & 0 \\
0 & 2 & 2 & 5
\end{pmatrix},\\
b = (50, 100, 90)
$$

### Solving our problem in Python 


In [ ]:
from scipy.optimize import linprog

vec_c = -np.array([12, 14, 25, 40]) # We want to maximize therefore minimize the negative
matrix_A = np.array([[1, 1, 1, 1], [3, 2, 1, 0],[0, 2, 2, 4]])
vec_b = np.array([50, 100, 90])
opt = linprog(c=vec_c, A_ub=matrix_A, b_ub=vec_b)
print(opt.x)

Maximum profits are achieved by selling 27.5 units of product 1 and 22.5 of product 4, and none for products 2 and 3!

### SciPy offers many special functions

- Airy, Error, Gamma, Kelvin, Lambert W, Legendre, Mathieu, Struve functions
- Elliptic functions and integrals
- Bessel functions
    - their zeros, integrals and derivatives
    - spherical Bessel and Ricatti-Bessel functions
- Statistical functions
    - Binomial, Beta, F, Gamma, Normal, Poisson, Student t, Kolmogorov, Chi square, ...
- Orthogonal polynomials (Jacobi, Hermite, Laguerre
- Combinatics
- ...





### Plotting Bessel function of the first kind - order 0
A simple example:

In [ ]:
from scipy import special

x = np.linspace(0, 30, 200)
y = special.j0(x)
plt.plot(x, y)
plt.grid()

### Key Points
- SciPy has a broad array of capabilities for solving numerical problems
- It is built over NumPy's arrays functionality
- It can be used to solve linear programming problems and much more

### Further Reading
- The SciPy documentation is [here](https://docs.scipy.org/doc/scipy/reference/)

### What's Next?
- Complete the exercises in this associated exercise notebook [X-18-SciPy.ipynb](X-18-SciPy.ipynb)
- Next notebook is [N-19-Pandas.ipynb](N-19-Pandas.ipynb)